In [30]:
import sys
sys.path.append('../')

from preprocessing.imdb_datareader import IMDBDataReader
from preprocessing.smore_datareader import SmoreDataReader
from preprocessing.xgboost_transformer import  XGBoostTransformer
import numpy 

user_path = './ml-100k/u.user'
item_path = './ml-100k/u.item'
user_item = './ml-100k/u.data'
reader = IMDBDataReader()

user_item  = reader.read_user_item_rating(user_item)
users = reader.read_user_data(user_path)
items = reader.read_item_data(item_path)


train_user_item = user_item[:int(len(user_item)*0.8)]
test_user_item = user_item[int(len(user_item)*0.8):]


smorereader = SmoreDataReader(transformer.u_idx, transformer.i_idx, "rep_dw.txt")
smore_user_vector = smorereader.read_user_data("rep_dw.txt")

total_users = {} 
for k, v in users.items(): 
    if k in smore_user_vector: 
        total_users[k] = v+smore_user_vector[k]

transformer = XGBoostTransformer(users, items, train_user_item)
X_train, Y_train, _, _, nFeatures = transformer.get_feature_vectors(total_users, items, train_user_item)
X_test, Y_test,X_cold_test, Y_cold_test, nFeatures = transformer.get_feature_vectors(total_users, items, test_user_item)

In [31]:
!aws s3api create-bucket --bucket recommendation-demo-yianc-0814 --region us-west-2 --create-bucket-configuration LocationConstraint=us-west-2


An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [32]:
bucket = 'recommendation-demo-yianc'
prefix = 'sagemaker/xgboost-movielens'
train_key      = 'train'
test_key       = 'test'
test_cold_key  = 'test_cold'


import io,boto3
import sagemaker.amazon.common as smac

def writeToLocalCSV(X, key, Y): 
    f_name = key + ".csv"
    output = open(f_name, 'w')


    for i in range(0, len(X)): 
        line = ''
        raw = X[i]
        line += str(int(Y[i])) 
        line += ' '
        for ele in raw:
            line += str(ele[0])+':'+str(float(ele[1]))+' '
        line += '\n'
        output.write(line)    
    return f_name  


 

train_data = writeToLocalCSV(X_train, train_key, Y_train)    
test_data = writeToLocalCSV(X_test, test_key, Y_test)    
test_cold_data = writeToLocalCSV(X_cold_test, test_cold_key, Y_cold_test)    
  
print('Output: {}'.format(output_prefix))

Output: s3://recommendation-demo-yianc/sagemaker/xgboost-movielens/output


In [33]:
!pip install xgboost

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [34]:
import xgboost as xgb
dtrain = xgb.DMatrix('train.csv')
test = xgb.DMatrix('test.csv')
test_cold = xgb.DMatrix('test_cold.csv')



param = {
    'max_depth': 5,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 2}  # the number of classes that exist in this datset
num_round = 3000  # the number of training iterations
bst = xgb.train(param, dtrain, num_round)

# dtrain = xgb.DMatrix('train.protobuf.csv')

preds = bst.predict(dtrain)


[02:55:07] 80000x109 matrix with 2843231 entries loaded from train.csv
[02:55:07] 2863x109 matrix with 102181 entries loaded from test.csv
[02:55:07] 89x109 matrix with 3216 entries loaded from test_cold.csv
[02:55:07] WARNING: /workspace/src/learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [26]:
from sklearn.metrics import accuracy_score

preds = bst.predict(test)
predictions = []
for i in range(0, len(preds)):
    if preds[i][0] > preds[i][1]:
        predictions.append(0)
    else:
        predictions.append(1)


In [27]:
accuracy = accuracy_score(Y_test, predictions)
accuracy

0.6685295144952846

In [28]:

preds = bst.predict(test_cold)
predictions = []
for i in range(0, len(preds)):
    if preds[i][0] > preds[i][1]:
        predictions.append(0)
    else:
        predictions.append(1)

In [29]:
accuracy = accuracy_score(Y_cold_test, predictions)
accuracy

0.7078651685393258